In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps

# Import API key
from api_keys import g_key 

In [34]:
#import csv and then create heatmap. Look up syuntax from heatnmap actviity.
#Use .loc function in order to filter out the df for ideal weather
#Create hotel_df and add column called "Hotel Name"
#Run google API calls and after I can refer googledat activity
#From there I can use that to get api calls for the hotel name in those cities
#Use the same heatmap and just add marker layer -- Look in the 3rd day actvoty folder


In [35]:
#Store filepath in a variable
city_data = "../cities.csv"

In [36]:
#Read data file woth pandas library
city_data_df = pd.read_csv(city_data, encoding="ISO-8859-1")

In [37]:
#Show header
city_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [38]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [39]:
#Use .loc function in order to filter our humidity, lat, and lng
#city_data_df.loc[:, ["Humidity", "Lat", "Lng"]]


In [40]:
#Narrow down the DataFrame to find your personal ideal weather conditions for a vacation.
temp_ideal = city_data_df.loc[(city_data_df["Max Temp"]>=65)&(city_data_df["Max Temp"]<=80)]
wind_ideal = temp_ideal.loc[(temp_ideal["Wind Speed"]<=5)]
weather_ideal_df = wind_ideal.loc[(wind_ideal["Cloudiness"]==1)]
weather_ideal_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
231,231,mocuba,1,MZ,1558378804,90,-16.84,36.99,65.76,2.42
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07


In [41]:
#Create hotel_df and add column "Hotel Name"
#Add "Hotel Name" to created dataframe and rename "hotel_df"
# create hotel file
hotel_df = weather_ideal_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,231,mocuba,1,MZ,1558378804,90,-16.84,36.99,65.76,2.42,
1,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07,


In [42]:
#Establish parameters for heatmap
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

In [43]:
#Create for statment to iterate through lat and long to search for hotels
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Query API url
    query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request
    hotel_search = requests.get(query_url, params=params)
    
    # convert to json
    hotel_search = hotel_search.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_df["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,231,mocuba,1,MZ,1558378804,90,-16.84,36.99,65.76,2.42,
1,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07,


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [45]:
 # Add marker layer ontop of heat map

# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

# Display figure

NameError: name 'heat_layer' is not defined